In [ ]:
# NOTE find out 10k most frequent tokens, assign them an index

frequency_dict = {}

with open('wiki2.train.txt', 'r') as f:
    train_data = f.read()

    for ind, token in enumerate(train_data.split()):
        try:
            frequency_dict[token] += 1
        except KeyError:
            frequency_dict[token] = 1

freq_lst = []
for key, value in frequency_dict.items():
    freq_lst.append(value)

freq_lst.sort()

freq_cutoff = freq_lst[-10000] # 16

# dictionary for turning token to integer
reduced_token_dict = {
}

# ensure <unk> token is above this threshold, which it is
print(frequency_dict['<unk>'])

ind = 0
for key, value in frequency_dict.items():
    if value > 16:
        reduced_token_dict[key] = ind
        ind += 1

print(f'Vocabulary size: {len(reduced_token_dict)}')


import pickle

with open(f'tokenizer.pkl', 'wb') as f:
    pickle.dump(reduced_token_dict, f)


54625
Vocabular size: 9892


In [ ]:
# NOTE tokenizer train, valid, and test

def tokenize(text, debug=False):
    inds = []
    with open(f'tokenizer.pkl', 'rb') as f:
        tokenizer_dict = pickle.load(f)
        unk_ind = tokenizer_dict['<unk>']
        if not debug:
            for token in text.split():
                try:
                    ind = tokenizer_dict[token]
                except KeyError:
                    ind = unk_ind

                inds.append(ind)
        if debug:
            for ind, token in enumerate(text.split()):
                try:
                    ind = tokenizer_dict[token]
                except KeyError:
                    ind = unk_ind
                print(f'token: {token} ind: {ind}')
                if ind > 15:
                    break
                inds.append(ind)
            
    return inds

for split in ['train', 'valid', 'test']:
    with open(f'wiki2.{split}.txt', 'r') as f:
        data = f.read()
        tokenized_data = tokenize(text=data, debug=False)
    with open(f'wiki2tokenized.{split}.pkl', 'wb') as f:
        pickle.dump(tokenized_data, f)


In [ ]:
embeddings = torch.nn.Embedding(
    num_embeddings=10, 
    embedding_dim=3
)

input_vec = torch.LongTensor([[1, 1, 4, 6, 7, 8]])

print('embedding weights:')
print(embeddings.weight)
print('embedding output:')
result = embeddings(input_vec)
print(result)
print(result.shape)

embedding weights:
Parameter containing:
tensor([[-0.9709,  0.1141, -0.3800],
        [ 1.3828,  0.4958,  0.4469],
        [ 0.9896, -1.8961, -0.3284],
        [ 0.6834, -0.2305,  0.2266],
        [ 0.7955, -1.2107, -0.6188],
        [ 0.7566, -2.0289, -1.1555],
        [ 0.2532,  0.2278,  0.0094],
        [-2.4239,  0.4996, -0.0987],
        [ 0.3315, -0.8013, -0.1278],
        [-0.1522,  0.9411,  1.4431]], requires_grad=True)
embedding output:


IndexError: index out of range in self

In [75]:
from torcheval.metrics.text import Perplexity
metric = Perplexity()
input = torch.tensor([[[0.3659, 0.7025, 0.3104]], [[0.0097, 0.6577, 0.1947]],[[0.5659, 0.0025, 0.0104]], [[0.9097, 0.0577, 0.7947]]])
target = torch.tensor([[2],  [1], [2],  [1]])
print(input.shape)
print(target.shape)
metric.update(input, target)
metric.compute()

torch.Size([4, 1, 3])
torch.Size([4, 1])


tensor(3.5257, dtype=torch.float64)